# Classification de formes à l'aide d'un réseau de convolution : Transfer Learning

> Author: Françoise Bouvet (IJCLab, CNRS)  
> Email: <francoise.bouvet@ijclab.in2p3.fr>

L'objectif de ce TP est de montrer le mécanisme de "transfer learning" sur un exemple simple.  
Le réseau utilisé est VGG16, préalablement entraîné sur ImageNet.  
They were randomly taken from a dataset provided on the kaggle platform database [kaggle](https://www.kaggle.com/).  
We use the [keras](https://keras.io/getting_started/) library.

#### Lecture des données (3 canaux)

In [ ]:
import numpy as np
from keras.utils import to_categorical
from utils import lecture_shape_3channels

rep_data = "../datasets/data_shape/"
lst_shape = ['circle', 'ellipse', 'rectangle', 'square', 'triangle']

# Read input data : input data are assumed to be represented on 3 channels
input_train_raw, output_train_raw = lecture_shape_3channels(rep_data + "train/", "*.png", lst_shape)

if input_train_raw is None or not np.any(input_train_raw):
    print(f'No file {extension} found in {dir}')

In [ ]:
from keras.utils import to_categorical

# Transform output into one hot encoding
output_train = to_categorical(output_train_raw)

#### Normalisation des données en entrée avec le prétraitement prévu pour VGG16

In [ ]:
from keras.applications.vgg16 import preprocess_input

# Preprocessing input ; scaling on [-1 ; 1]
def normalize(data):
    return (2. * np.array(data).astype('float32') / 255.) - 1

# Normalize input data
input_train = normalize(input_train_raw)

In [ ]:
# Shuffle input data
ind = np.arange(0, np.shape(input_train)[0])
np.random.shuffle(ind)

# Apply to data ; 
input_train = input_train[ind]
output_train = output_train[ind]

print(f'The dataset contains {input_train.shape[0]} samples')

#### Lecture du réseau préentrainé VGG16 et initialisation de notre modèle  

In [ ]:

from keras.models import Sequential
from keras.layers import Flatten,Dense, Dropout
from keras.applications.vgg16 import VGG16

base_model = VGG16(include_top=False, weights='imagenet',
                   input_shape=(64, 64, 3), pooling='max', classes=5)

model = Sequential()

# The convolution layers are set immutable
for layer in base_model.layers:
   layer.trainable = False
   model.add(layer)

#### La partie MLP du réseau est la partie entrainable.

In [ ]:
model.add(Flatten())

# Full connected layer (MLP)
# Only these layers are trainable
# Regularisation mechanism like droput can be added
...

In [ ]:
# Compilation
model.compile(...)

# Display the model
model.summary()

#### Apprentissage

In [ ]:
from keras.callbacks import ReduceLROnPlateau

# Decrease the learning rate factor
reduce_lr = ReduceLROnPlateau(...)

In [ ]:
history = model.fit(...)

In [ ]:
from utils import draw_history

draw_history(history)

#### Evaluation

In [ ]:
# Read test data
input_test_raw, output_test_raw = lecture_shape_3channels(rep_data + "test/", "*.png", lst_shape)


In [ ]:
# Normalize input data
  
input_test = normalize(np.array(input_test_raw))

# Transform output into one hot encoding
output_test = to_categorical(np.array(output_test_raw))

# Test set shuffle only aims to display samples from all the classes
ind = np.arange(np.shape(input_test)[0])
np.random.shuffle(ind)
input_test = np.array(input_test)[ind]
output_test =np.array(output_test)[ind]

# Evaluate the model ; the two parameters are the input_test array and the output_test array
sum_score = model.evaluate(input_test, output_test)
print("Data test : loss %.3f accuracy %.3f" % (sum_score[0], sum_score[1]))


In [ ]:
# Display prediction and true class for some samples
output_predict = model.predict(input_test)
for sample_predict, sample_true in zip(output_predict[0:5], output_test[0:5]):
    print(sample_predict, sample_true)

In [ ]:
from utils import draw_multiple_images

# Display some images
n=min(40, np.shape(input_test)[0])
nb_col = 8
image_display = input_test[0:n, :, : , 1].reshape(n, 64, 64, 1)
draw_multiple_images(image_display, output_test[0:n], output_predict[0:n], 
                     lst_shape, nb_col, np.shape(image_display)[1:])